In [1]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
''
enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

import sys
sys.path.append('../')
from config import Config, Shape
from plate import get_base, get_screw_positions, get_key_positions, make_plate
from case import get_basic_shape,  cut_aviator_connector_hole, get_distance_between_two_vertices, make_controller_box_top_plate, add_microcontrollerbox

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


/home/ben/anaconda3/envs/jcq3/lib/python3.10/site-packages/quaternion/calculus.py:310: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def fd_indefinite_integral(f, t):


In [5]:

rkn = [4,8,8,6]
config=Config(rkn, shape=Shape.LEAN, floorThickness=3)
kp = get_key_positions(config)

In [47]:
case = make_case(config)

cc = make_controller_box_top_plate(case, config)

#cc=cc.faces('>Z').workplane(centerOption="CenterOfBoundBox").hole(3)

#case = case.union(cc).clean()

#case.edges('|Z').fillet(config.edgeFillet)


case.faces(tag='controllerBoxTop')#.vertices()

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.29s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [16]:
get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<Y'))

{'x': 56.0, 'y': 0.0, 'z': 0.0}

In [17]:
get_distance_between_two_vertices(cc.faces('>Z').vertices('<Y'))

{'x': 56.0, 'y': 0.0, 'z': 0.0}

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.00s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [38]:
if True:
    case= get_basic_shape(config)
    case.faces('<Z').tag('bottomface')

    # add box for microcontroller
    if config.controller is not None:
        case = add_microcontrollerbox(case, config)
    

case.faces(tag='bottomface').shell(-config.wallThickness)

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [39]:


def make_case(config:Config) -> cq.Sketch:
    """TODO: add screw holes for microcontroller (check they are in right position!!)
            change fillet parameters
            add module for leds, encoder and aviator
            """
    
    
    case= get_basic_shape(config)
    case.faces('<Z or >Z').tag('bottomface')

    # add box for microcontroller
    if config.controller is not None:
        case = add_microcontrollerbox(case, config)
    

    #fillet edges
    case = case.edges('|Z').fillet(config.edgeFillet)
    #case.edges('>Z').tag('outerTopEdge')


    # scoop out interior
    case= (case.faces(">Z or <Z").shell(-config.wallThickness, kind='intersection'))
    
    # set floor thickness 
    case = case.faces('<Z').wires('>X and >Y').toPending().extrude(config.floorThickness)
    
    #fillet bottom edge
    case = case.edges('<Z').fillet(3)



    # add plate screw holes 
    scr_hls = get_screw_positions(config)
    case=(case.faces('<Z').workplane(origin=(0,0))
          .pushPoints([(x,-y) for x,y in scr_hls])
          .cskHole(2.4, 4.8, 82, depth=None)    
          )

    # add controllerbox holes.
    # TODO - remove hardcoding on screw hole size
    if config.controller is not None:
        case = (case.workplaneFromTagged("controllerBox")
                .rect(config.controller.screw_hole_x,
                        config.controller.screw_hole_y- config.wallThickness - config.controllerYOffset - (config.controller.board_dimension_y -config.controller.screw_hole_y)*0.5, 
                        forConstruction=True ,centered=[True,True,True])
                .vertices()
                .cskHole(2.4, 4.8, 82, depth=None))
    
    #cut hole for aviator connector 
    case = cut_aviator_connector_hole(case, config)

    return case


In [11]:
case= get_basic_shape(config)
    #case.faces('<Z').tag('bottomface')

    # add box for microcontroller
if config.controller is not None:
        case = add_microcontrollerbox(case, config)
    
case= (case.faces(">Z or <Z").shell(-config.wallThickness, kind='intersection'))



In [25]:
casse = case.edges('|Z').fillet(2)

{'x': 56.0, 'y': 0.0, 'z': 0.0}